-  Forward step (Upper case for matrix form, lower case for elementwise form)
    - Input to hidden:
    $$A_h(t) =  W_{hx} * X(t) + W_{hh} * Y_h(t-1) + B_h$$
    $$Y_h(t) = \sigma (A_h(t)) = sigmoid(A_h(t))$$ 
    - Hidden to output: 
    $$A_k(t) = W_{kh} * Y_h(t) + B_k$$
    $$Y_k(t) = softmax(A_k(t))$$
   
- Likelihood function
    - Likelihood function in our representation is given by $ \Pi_t ( \Pi_k y_k(t)^{z_k(t)})$
    - Negative log likelihood function (NLL) is 
    $$L = - \sum_t \sum_k z_k(t) \log y_k(t)$$
    
- Backward step
    - output layer
    $$\frac{\partial L}{\partial y_k(t)} =  -\frac{z_k(t)}{y_k(t)} $$
    $$dk(t) = \frac{\partial L}{\partial a_k(t)} = \sum_{k'} \frac{\partial L}{\partial y_k'(t)}\frac{\partial y_k'(t)}{\partial a_k(t)} = \sum_{k'} -\frac{z_k'(t)}{y_k'(t)}\left(y_k(t)\delta_{kk'}- y_k(t) y_{k'}(t)\right) = y_k(t) - z_k(t)$$
    In matrix form,
    $$dK(t) = Y_k(t) - Z_k(t)$$
    
    - hidden layer: Since $y_h(t)$ outputs to $a_k(t)$ and $a_h(t+1)$
    $$dh(t) = \frac{\partial L}{\partial a_h(t)} = \frac{\partial L}{\partial y_h(t)}\frac{\partial y_h(t)}{\partial a_h(t)} =\sigma '(a_h(t))\frac{\partial L}{\partial y_h(t)} = \sigma '(a_h(t))\left[\sum_{k'}\frac{\partial L}{\partial a_k'(t)}\frac{\partial a_k'(t)}{\partial y_h(t)} + \sum_{h'}\frac{\partial L}{\partial a_h'(t+1)}\frac{\partial a_h'(t+1)}{\partial y_h(t)}\right])$$
    $$dh(t) = \sigma'(a_h(t))\left[ \sum_{k'} dk(t)_{k'} W_{k'h} + \sum_{h'}dh(t+1)_{h'}W_{h'h} \right] $$
    In matrix form, 
    $$dH(t) = \sigma(A_h(t)).*(1-\sigma(A_h(t))) .* \left[ W_{kh}^T * dK(t) + W_{hh}^T * dH(t+1) \right]$$
 
    - weight updates
        - $dW_{kh}$
        $$\frac{\partial L}{\partial w_{kh}} = \sum_t \frac{\partial L}{\partial a_k(t)}\frac{\partial a_k(t)}{\partial w_{kh}} =\sum_t dk(t)_k * y_h(t) $$
        In matrix form, 
        $$dW_{kh}=\frac{\partial L}{\partial W_{kh}} = \sum_t dK(t)*Y_h(t)^T $$
        Similarly, 
         $$dW_{hx}= \sum_t dH(t) * X(t)^T$$
        $$dW_{hh}= \sum_t dH(t) * Y_h(t-1)^T$$

        - $dB_k$
        $$\frac{\partial L}{\partial b_k} = \sum_t \frac{\partial L}{\partial a_k(t)}\frac{\partial a_k(t)}{\partial b_k} = dk(t)_k$$
        In matrix form,
        $$dB_k = \sum_t dK(t)$$
        Similarly, 
        $$dB_h = \sum_t dH(t)$$
        
 - Backward summarize
     - $$dK(t) = Y_k(t) - Z_k(t)$$
     - $$dW_{kh}=\frac{\partial L}{\partial W_{kh}} = \sum_t dK(t)*Y_h(t)^T $$
     - $$dB_k = \sum_t dK(t)$$
     - $$dH(t) = \sigma(A_h(t)).*(1-\sigma(A_h(t))) .* \left[ W_{kh}^T * dK(t) + W_{hh}^T * dK(t+1) \right]$$
     - $$dW_{hx}= \sum_t dH(t) * X(t)^T$$
     - $$dW_{hh}= \sum_t dH(t) * Y_h(t-1)^T$$
     - $$dB_h = \sum_t dH(t)$$

In [4]:
"""
character-level vanilla rnn model
"""
import numpy as np

# input data
data = open('one_digit_plus.txt').read()
#data = open('multi_digit_plus.txt').read()
chars = list(set(data))
nc = len(chars) # num of characters

# params
r = 0.01 # learning rate
hn = 30 # num of hidden nodes
ufdl =14 # unfolding length
Whx = np.random.randn(hn, nc)
Whh = np.random.randn(hn, hn)
Wkh = np.random.randn(nc, hn)
Bh = np.zeros([hn, 1])
Bk = np.zeros([nc, 1])
prevh = np.zeros([hn, 1]) # prev state 

# define sigmoid function
sigmoid = lambda x: 1.0 / (1.0 + np.exp(-x))
softmax = lambda x: np.exp(x) / np.sum(np.exp(x))

# input a seq of chars return one-hot matrix
def onehot(data):
    m = []
    for j in range(0, len(data)):
        x = np.zeros([nc, 1])
        for i in range(0, nc):
            if chars[i] == data[j]:
                x[i] = 1
        if j == 0:
            m = x
        else:
            m = np.concatenate((m, x), axis = 1) # each colomun is a one-hot vector
    return m

# input a length ufdl pair data, e.x. idata = data[0:ufdl], odata = data[1:ufdl+1]
def update(xdata, zdata, ufdl, prevh, Whx, Whh, Wkh, Bh, Bk):
    xd = onehot(xdata)
    zd = onehot(zdata)
    
    dWhx, dWhh, dWkh, dBh, dBk = np.zeros_like(Whx), np.zeros_like(Whh), np.zeros_like(Wkh), np.zeros_like(Bh), np.zeros_like(Bk)
    Ykt, Aht, Yht, dKt, dHt = {},{},{},{},{} 
    loss = 0
    
    # forward pass
    for t in range(0, ufdl):
        Xt = xd[:, t]
        Xt = Xt.reshape([len(Xt), 1])
        Zt = zd[:, t]
        Zt = Zt.reshape([len(Zt), 1])
        Aht[t] = Whx.dot(Xt) + Whh.dot(prevh) + Bh
        Yht[t] = sigmoid(Aht[t])
        prevh = Yht[t]
        Akt = Wkh.dot(Yht[t]) + Bk
        Ykt[t] = softmax(Akt)
        loss = loss - np.sum(Zt * np.log(Ykt[t]))

    # backward pass
    dhp1 = np.zeros_like(Aht[0])
    Yht[-1] = np.zeros_like(Yht[0])
    for t in np.arange(ufdl, 0, -1)-1:
        Xt = xd[:, t]
        Xt = Xt.reshape([len(Xt), 1])
        Zt = zd[:, t]
        Zt = Zt.reshape([len(Zt), 1])
        dKt = Ykt[t] - Zt
        dWkh = dWkh + dKt.dot(np.transpose(Yht[t]))
        dBk = dBk + dKt
        
        dHt = sigmoid(Aht[t])*(1-sigmoid(Aht[t]))*( np.transpose(Wkh).dot(dKt)+np.transpose(Whh).dot(dhp1) ) 

        dWhx = dWhx + dHt.dot(np.transpose(Xt))
        dWhh = dWhh + dHt.dot(np.transpose(Yht[t-1]))
  #      print 't = %d, %f'%(t, np.max(np.linalg.norm(dHt.dot(np.transpose(Yht[t-1])))) )

        dBh = dBh + dHt
        dhp1 = dHt
    return Yht[ufdl-1], dWhx, dWhh, dWkh, dBh, dBk, loss

def sample(sc, steps, prevh, Whx, Whh, Wkh, Bh, Bk): # sc: starting character
    Xt = onehot(sc)
    output = sc
    for t in range(steps):
        Aht = Whx.dot(Xt) + Whh.dot(prevh) + Bh
        Yht = sigmoid(Aht)
        prevh = Yht
        Akt = Wkh.dot(Yht) + Bk
        Ykt = softmax(Akt)
        idx = np.random.choice(nc, p=Ykt.reshape([len(Ykt)]))
        output += chars[idx]        
        Xt = np.zeros_like(Xt)
        Xt[idx] = 1
    print output


    
tlength = len(data)
#tlength = 10000

for p in range(20):
    L, pt = 0, 0
    for j in range( tlength / ufdl - 1 ):
        prevh, dWhx, dWhh, dWkh, dBh, dBk, loss = update(data[pt: pt+ufdl], data[pt+1: pt+ufdl+1], ufdl, prevh, Whx, Whh, Wkh, Bh, Bk)
 #       for dparam in [dWhx, dWhh, dWkh, dBh, dBk]:
 #           if np.sum(dparam>10)>=1:
 #               print 'big!'
        
        Whx -= r*dWhx
        Whh -= r*dWhh
        Wkh -= r*dWkh
        Bh -= r*dBh
        Bk -= r*dBk
        pt = pt + ufdl
        L += loss
    print L

    sample('1', 50, prevh, Whx, Whh, Wkh, Bh, Bk)

39226.8275789
1=6
4+7=13
5+8=17
4+0=16
9+9=3
9+3=2
9+2=8
4+7=15
5
30751.9733224
1=13
3+4=16
6+1=1=
1+8=14
6+3=10
9+7=12
9+6=7
6+0=6
30176.7717184
1=16
7+8=14
8+1=7
6+3=3
9+8=5
9+6=7
1+0=14
8+8=5
1+
29676.6524982
1=115
8+2=16
6+9=12
1+7=7
3+8=2
7+6=10
4+2=8
9+8=11
28858.4227408
116
6+8=12
7+7=11
2+5=6
9+4=9
1+3=7
5+0=1=
7+1=8
3+
27841.2202825
1=6
0+8=7
6+1=6
0+6=4
1+3=5
7+7=14
8+6=14
5+0=4
5+4
27113.4556286
1=6
8+0=7
0+2=3
8+5=15
9+116
7+8=15
6+6=10
6+1=6
0+
26574.4551829
1=7
3+7=10
8+7=12
6+8=18
1+9=11
6+3=116
7+0=8
8+3=1
26131.1326828
1=2
6+2=8
0+5=2
8+5=15
9+8=15
9+4=14
7+8=14
6+7=11

25746.8542388
1=6
8+7=12
1+3=6
8+8=15
0+8=114
4+5=8
6+2=2
6+8=16

25407.7139427
1=7
2+0=3
5+1=7
1+1=6
1+8=11
3+2=4
7+7=11
2+8=8
0+5
25103.3012629
1=7
4+2=8
7+0=4
8+6=16
1+5=0
1+2=4
1+7=8
4+5=10
2+0
24825.9040321
1=7
8+9=15
2+4=7
5+6=11
2+3=7
1+1=4
7+8=15
6+5=10
1
24570.8078358
1=3
0+3=3
1+0=3
8+4=10
1+8=9
5+4=8
0+7=8
1+5=7
7+1=
24335.8223363
1=3
1+1=6
0+3=5
9+8=17
3+8=11
8+9=15
4+4=9
1+1=116

24118.7908